# Lab 7: GPU Programming Introduction
**Objectives:**

Learn about GPU architecture and CUDA programming.

Understand the advantages of parallel processing on GPUs.

Write and execute basic CUDA programs.

In [ ]:
!g++ --version


g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



# How to write C++ files in Collab

In [ ]:
%%writefile program.cpp
#include <iostream>
using namespace std;

int main() {
    cout << "Hello, C++ in Google Colab!" << endl;
    return 0;
}


Writing program.cpp


# Compile and Execute

In [ ]:
!g++ program.cpp -o program
!./program


Hello, C++ in Google Colab!


# Check GPU Version

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# Writing and Running a CUDA Program

Use the *%%writefile* to create a .cu file, compile it with nvcc, and run it.






In [ ]:
%%writefile first_cuda_program.cu
#include <iostream>
// CUDA kernel to perform addition on the GPU
__global__ void addGPU(int a, int b, int* result_gpu) {
    *result_gpu = a + b;
}
int main() {
    // CPU variables
    int a = 5;
    int b = 7;
    int result_gpu = 0;

    int* d_result_gpu; // Pointer to store the GPU result

    // Allocate GPU memory for the result
    cudaMalloc((void**)&d_result_gpu, sizeof(int));

    // Launch GPU kernel
    addGPU<<<1, 1>>>(a, b, d_result_gpu);

    // Copy result from GPU to CPU
    cudaMemcpy(&result_gpu, d_result_gpu, sizeof(int), cudaMemcpyDeviceToHost);

    // Free GPU memory
    cudaFree(d_result_gpu);

    std::cout << "CPU Result: " << a + b << std::endl;
    std::cout << "GPU Result: " << result_gpu << std::endl;

    return 0;
}



Overwriting first_cuda_program.cu


In [ ]:
!nvcc first_cuda_program.cu -o out
!./out

CPU Result: 12
GPU Result: 12


**Example:** Vector Addition in CUDA

In [ ]:
%%writefile vector_add.cu
#include <cuda.h>
#include <stdio.h>

__global__ void vectorAdd(int *a, int *b, int *c, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 1024;
    int size = n * sizeof(int);

    int *h_a, *h_b, *h_c; // Host pointers
    int *d_a, *d_b, *d_c; // Device pointers

    // Allocate memory on the host
    h_a = (int *)malloc(size);
    h_b = (int *)malloc(size);
    h_c = (int *)malloc(size);

    // Initialize host arrays
    for (int i = 0; i < n; i++) {
        h_a[i] = i;
        h_b[i] = i;
    }

    // Allocate memory on the device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Launch kernel
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    // Copy result back to host
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    // Print result
    for (int i = 0; i < 10; i++) {
        printf("%d ", h_c[i]);
    }

    // Free memory
    free(h_a); free(h_b); free(h_c);
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);

    return 0;
}


Writing vector_add.cu


In [ ]:
!nvcc vector_add.cu -o vector_add
!./vector_add


0 2 4 6 8 10 12 14 16 18 